# Check The Rotor troubleshooting 

In [ ]:
import os
import subprocess

import pandas as pd

from arc.job.trsh import scan_quality_check, trsh_scan_job, determine_ess_status
from arc.parser import parse_scan_args, parse_1d_scan_energies, parse_geometry, parse_scan_conformers
from arc.plotter import plot_1d_rotor_scan

from arc.species.converter import xyz_to_xyz_file_format

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
path = '/Users/xiaorui/Dropbox/RMG/Co-OPTIMA/Hyperboost/Weird rotor/input.log'

## 1.Check Convergence
'done' means normally converged

In [ ]:
determine_ess_status(path, '', 'scan', 'gaussian')

## 2. Scan Quality check
`False` means nothing bad, and `True` means something wrong and needs troubleshooting

In [ ]:
scan_args = parse_scan_args(path)
energies, angles = parse_1d_scan_energies(path) 
quality = scan_quality_check('', scan_args['scan'][1:-1], energies, scan_args['step_size'], [], path)
print('scan: ', scan_args['scan'], 'scan_res:', scan_args['step_size'])
quality

## 3. Scan troubleshooting

In [ ]:
if 'freeze' in quality[3]:
    species_scan_lists = [scan_args['scan']]
    solution = trsh_scan_job('', scan_args['step_size'], scan_args['scan'], species_scan_lists, quality[3], path)
    print(solution[0])

if 'change conformer' in quality[3]:
    xyz = parse_geometry(path)
    new_xyz = quality[3]['change conformer']
    print('original:')
    print(xyz_to_xyz_file_format(xyz))
    print('New')
    print(xyz_to_xyz_file_format(new_xyz))

## 4. Check curve

In [ ]:
plot_1d_rotor_scan(angles, energies)

## 5. Open Molden
Exit molden either in a normal way (skull button) or stop the kernel

In [ ]:
cmd = ['/Users/xiaorui/Apps/molden/molden', f'{path}']
subprocess.Popen(cmd)

## 6. Look at values of internal coordinates

In [ ]:
scan_conformers = parse_scan_conformers(path)

you can apply `DataFrame` type filter to search for internal coordinates values, here shows an example

In [ ]:
scan_conformers[(scan_conformers['type'] == 'D') & (scan_conformers['scan'] == False)].loc[:, ['atoms', 0] + list(range(7,46))]